数据分析师和科学家经常在数据规整和准备上花费大量的时间，这里也反映了精通这些技术的重要性。

使用哪个库进行模型开发取决于应用。很多统计上的难题可以通过更简单的技术，例如最小二乘回归等方法，来解决，但另外一些问题可能需要用到高阶的机器学习方法。幸运的是Python已经成为实现分析方法的语言选择之一，因此在完成本书学习后可以探索很多工具的使用。

在本章，将回顾pandas的一些特性，这些特性可能会在使用pandas进行模型训练和评分时有用。将介绍两个流行的建模工具包——statsmodels（http://statsmodels.org ）和scikit-learn（http://scikit-learn.org ）。由于这两个项目每个都大到可以单独成书，就不再尝试完整地介绍，而是像其他基于Python的数据科学、统计学和机器学习的书籍一样，直接进入到项目的官方在线文档。

In [1]:
import numpy as np
import pandas as pd

## 13.1　pandas与建模代码的结合
使用pandas用于数据载入和数据清洗，之后切换到模型库去建立模型是一个常见的模型开发工作流。在机器学习中，特征工程是模型开发的重要部分之一。特征工程是指从原生数据集中提取可用于模型上下文的有效信息的数据转换过程或分析。

尽管一个“好”的特征工程的细节已经超出了本书的范围，但仍然会展示一些可以在利用pandas进行数据操作和建模之间无痛切换的方法。

pandas和其他分析库的结合点通常是NumPy数组。要将DataFrame转换为NumPy数组，使用.values属性：

In [2]:
data = pd.DataFrame({
        'x0': [1, 2, 3, 4, 5],
        'x1': [0.01, -0.01, 0.25, -4.1, 0.],
        'y': [-1.5, 0., 3.6, 1.3, -2.]})

data   

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [10]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [11]:
data.values.dtype

dtype('float64')

将数组再转换为DataFrame，可以传递一个含有列名的二维ndarray：

In [5]:
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])

df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


> .values属性一般在数据是同构化的时候使用——例如，都是数字类型的时候。如果数据是异构化的，结果将是Python对象(object)的ndarray：

In [6]:
df3 = data.copy()

df3['strings'] = ['a', 'b', 'c', 'd', 'e']

df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [7]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

对于某些模型，可能只想使用一部分列。推荐使用`loc`索引和`values`：

In [13]:
model_cols = ['x0', 'x1']

data.loc[:, model_cols].values

,x0,x1
0,1,0.01
1,2,-0.01
2,3,0.25
3,4,-4.10
4,5,0.00


有些库对pandas有本地化支持，可以自动做以下工作：将数据从DataFrame转换到NumPy中并将模型参数名称附于输出表的列或Series上。在其他情况下，将不得不手动去处理这些“元数据管理”的操作。

在第12章，学习了pandas的Categorical类型和pandas.get_dummies函数。假设示例数据集中，有一个非数字类型的列：

In [14]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])

data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


如果想使用虚拟变量替代'category'列，先创建虚拟变量，之后删除'categroy'列，然后连接结果：

In [15]:
dummies = pd.get_dummies(data.category, prefix='category')
data_with_dummies = data.drop('category', axis=1).join(dummies)

data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


在使用虚拟变量拟合特定的统计模型时是有一些细微区别的。当拥有不止简单的数字类型列时，使用`Patsy`可以更简单、更少出错。

## 13.2　使用Patsy创建模型描述
Patsy（https://patsy.readthedocs.io/ ）是一个用于描述统计模型（尤其是线性模型）的Python库。它使用一种小型基于字符串的“公式语法”，这种语法受到了R、S统计编程语言中公式语法的启发。

Patsy能够很好地支持statsmodels中特定的线性模型，因此将专注于它的主要特性。Patsy的公式是特殊字符串语法，如下：
```
y ~ x0 + x1
```
语法a+b并不代表a加b，而是指为模型而创建的设计矩阵中的名词列。`patsy.dmatrices`函数在数据集上（可以是一个DataFrame或数组的字典）使用了一个公式字符串，并为一个线性模型产生了设计矩阵：

In [16]:
data = pd.DataFrame({
        'x0': [1, 2, 3, 4, 5],
        'x1': [0.01, -0.01, 0.25, -4.1, 0.],
        'y': [-1.5, 0., 3.6, 1.3, -2.]})

data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [17]:
import patsy

y, X = patsy.dmatrices('y ~ x0 + x1', data)

现在可以得到：

In [18]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [19]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

这些Patsy的DesignMatrix实例是含有附加元数据的NumPy ndarray：

In [20]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [21]:
np.asarray(X)    

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

可能会猜想`Intercept`（截距）这个名词列的由来。这其实是线性模型，如最小二乘回归（OLS，Ordinary Least Squares）中的惯例。可以通过给模型添加名词列`+0`来加入截距：

In [22]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

Patsy对象可以直接传递给一些算法，比如`numpy.linalg.lstsq`等，这些算法都会执行一个最小二乘回归：

In [27]:
coef, resid, _, _ = np.linalg.lstsq(X, y, rcond=None)

模型元数据保留在`design_info`属性中，因此可以将模型列名重新附加到拟合系数以获得一个Series，例如：

In [28]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [29]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)

coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### 13.2.1　Patsy公式中的数据转换
可以将Python代码混合到Patsy公式中，在执行公式时，Patsy库将尝试在封闭作用域中寻找使用的函数：

In [30]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
# 结果为上面的数值的算式结果
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

一些常用的变量转换包括标准化（对均值0和方差1）和居中（减去平均值）。为了实现这个目的，Patsy具有内置函数：

In [32]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

作为建模的一部分过程，可能会在一个数据集上拟合一个模型，之后基于另一个模型评价该模型。这个过程可以保留部分数据或者之后再加入新数据。在应用像居中和标准化这样的转换时，在基于新数据使用模型进行预测的时候要小心。这些转换被称为有状态的转换，因为在形成新数据集时必须使用原数据集中的均值或标准差等统计值。

`patsy.build_design_matrices`函数可以使用原始样本内数据集中保存的信息将变换应用于新的样本外数据上：

In [33]:
new_data = pd.DataFrame({
        'x0': [6, 7, 8, 9],
        'x1': [3.1, -0.5, 0, 2.3],
        'y': [1, 2, 3, 4]})

new_X = patsy.build_design_matrices([X.design_info], new_data)

new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

因为加号`(+)`在Patsy公式的上下文中并不是加法的意思，当要对数据集中两列按列名相加时，必须将列名封装到特殊的`I`函数中：

In [34]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

在`patsy.builtins`模块中Patsy还有其他一些内建转换。更多内容请参看官方在线文档。

### 13.2.2　分类数据与Patsy
有多种方式可以将非数字类型数据转换以用于模型的设计矩阵。一个完整的解决方案是超出了范围的，最好是能够学习一门统计学课程。

当在Patsy公式中使用非数字名词列时，它们将会被默认转换为虚拟变量。如果有拦截，其中一个级别将被排除以避免共线性：

In [39]:
data = pd.DataFrame({
        'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
        'key2': [0, 1, 0, 1, 0, 1, 0, 0],
        'v1': [1, 2, 3, 4, 5, 6, 7, 8],
        'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
    })

y, X = patsy.dmatrices('v2 ~ key1', data)

X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

如果忽略了模型的截距，每个类别值的列将会被包含在模型的设计矩阵中：

In [40]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

数字类型列可以使用`C`函数解释为分类类型：

In [41]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

当在模型中使用多个分类名词列时，事情可能会更加复杂，因为可以包含形式为key1：key2的交互项，例如，可用于方差分析（ANOVA）模型：

In [42]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})

data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [46]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [47]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

# 感觉是与（and）操作
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

In [ ]:
Patsy提供了其他几种方式，可以转换分类数据，包括按照特定顺序对名词列进行转换。更多内容请参看官方在线文档。

## 13.3　statsmodels介绍
statsmodels（http://www.statsmodels.org ）是一个Python库，用于拟合多种统计模型，执行统计测试以及数据探索和可视化。statsmodels包含更多的“经典”频率学派统计方法，而贝叶斯方法和机器学习模型可在其他库中找到。

包含在statsmodels中的一些模型：

* 线性模型，广义线性模型和鲁棒线性模型

* 线性混合效应模型

* 方差分析（ANOVA）方法

* 时间序列过程和状态空间模型

* 广义的矩量法

### 13.3.1　评估线性模型
统计模型中有几种线性回归模型，从较基本的（例如，普通最小二乘）到更复杂的（例如，迭代重新加权的最小二乘）。

statsmodels中的线性模型有两个不同的主要接口：基于数组的和基于公式的。这些接口通过这些API模块导入来访问：

In [48]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

为了展示如何使用这些，将根据一些随机数据生成线性模型：

In [83]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

# 用于复现
np.random.seed(12345)
N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

在这里，写下了已知参数beta的“真实”模型。在这种情况下，dnorm是用于生成具有特定均值和方差的正态分布数据的辅助函数。所以现在有：

In [84]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [85]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

线性模型通常与在Patsy中看到的截距项相匹配。`sm.add_constant`函数可以将截距列添加到现有矩阵：

In [86]:
X_model = sm.add_constant(X)

X_model[:5]   

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

`sm.OLS`类可以拟合一个最小二乘线性回归：

In [87]:
model = sm.OLS(y, X)

模型的`fit`方法返回一个回归结果对象，该对象包含了估计的模型参数和其他的诊断：

In [88]:
results = model.fit()

results.params

array([0.17826108, 0.22303962, 0.50095093])

在results上调用`summary`方法可以打印出一个模型的诊断细节：

In [58]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Sun, 14 Jun 2020   Prob (F-statistic):                    7.44e-12
Time:                        16:18:33   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1783      0.053      3.364      0.001       0.073       0.283
x2             0.2230      0.046      4.818      0.000       0.131       0.315
x3             0.5010      0.080      6.237      0.000       0.342       0.660
==============================================================================
Omnibus:                        4.662   Durbin-Watson:                   2.201
Prob(Omnibus):                  0.097   Jarque-Bera (JB):                4.098
Skew:                           0.481   Prob(JB):                        0.129
Kurtosis:                       3.243   Cond. No.                         1.74
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [91]:
# 使用模型进行预测
results.predict(X[:5])

array([-0.04097923, -0.17052219,  0.00510113, -0.35250757, -0.12355709])

此处的参数名称已被赋予通用名称x1、x2等。假设所有模型参数都在DataFrame中：

In [60]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])

data['y'] = y

data.head()

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


现在可以使用statsmodels公式API和Patsy公式字符串：

In [61]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()

results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [62]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

观察statsmodels如何将结果作为带有DataFrame列名称的Series返回。使用公式和pandas对象时，也不需要使用add_constant。

给定新的样本外数据后，可以根据估计的模型参数计算预测值：

In [63]:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

还有很多额外的工具，可针对在statsmodels中能够探索的线性模型结果进行分析、诊断和可视化。除了普通最小二乘法之外，还有其他种类的线性模型。

### 13.3.2　评估时间序列处理
statsmodels中的另一类模型用于时间序列分析。其中包括自回归过程，卡尔曼滤波和其他状态空间模型，以及多变量自回归模型。

模拟一些具有自回归结构和噪声的时间序列数据：

In [92]:
init_x = 4

import random
values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

该数据具有参数为0.8和-0.4的AR(2)结构（两个滞后）。当拟合一个AR模型时，可能不知道包含的滞后项的数量，所以可以用更大的滞后数来拟合该模型：

In [103]:
MAXLAGS = 5

model = sm.tsa.AR(values) # 有弃用告警
# model = sm.tsa.SARIMAX(values)

/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/ar_model.py:691: FutureWarning: 
statsmodels.tsa.AR has been deprecated in favor of statsmodels.tsa.AutoReg and
statsmodels.tsa.SARIMAX.

AutoReg adds the ability to specify exogenous variables, include time trends,
and add seasonal dummies. The AutoReg API differs from AR since the model is
treated as immutable, and so the entire specification including the lag
length must be specified when creating the model. This change is too
substantial to incorporate into the existing AR api. The function
ar_select_order performs lag length selection for AutoReg models.

AutoReg only estimates parameters using conditional MLE (OLS). Use SARIMAX to
estimate ARX and related models using full MLE via the Kalman Filter.

To silence this warning and continue using AR until it is removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.ar_model.AR', FutureWarning)

  warnings.warn(AR_DEPRECATION_WARN, FutureWarning)


In [105]:
results = model.fit(MAXLAGS)

结果中的估计参数首先是截距，接下来是前两个滞后的估计：

In [106]:
results.params

array([-0.00616093,  0.78446347, -0.40847891, -0.01364148,  0.01496872,
        0.01429462])

这些模型的深层细节以及如何解释其结果超出了范围，但在statsmodels文档中还有很多可以发现的内容。

## 13.4　scikit-learn介绍
scikit-learn（http://scikit-learn.org ）是使用最广泛且最受信任的通用Python机器学习库。它包含广泛的标准监督的和无监督的机器学习方法，包括用于模型选择和评估、数据转换、数据加载和模型持久化的工具。这些模型可用于分类、聚类、预测和其他常见任务。

有很多优秀的在线和印刷资源可用于学习机器学习，以及如何应用scikit-learn和TensorFlow等库来解决实际问题。在本节中，简要介绍一下scikit-learn API风格。

在写这篇文章的时候，scikit-learn并没有深度地和pandas集成，尽管还有一些附加的第三方包仍在开发中。不过，在模型拟合之前，pandas对于“按摩”数据集非常有用。

作为一个例子，使用Kaggle比赛（https://www.kaggle.com/c/titanic ）中关于泰坦尼克号上生还乘客的经典数据集，其中泰坦尼克号于1912年沉没。使用pandas载入测试和训练数据集：

In [110]:
train = pd.read_csv('datasets/titanic/train.csv')

test = pd.read_csv('datasets/titanic/test.csv')

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


像statsmodels和scikit-learn通常不能提供缺失数据，因此要检查各列，看看是否有包含缺失数据：

In [111]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [115]:
# 反向检查缺失值，另外请注意descirbe()不能检查非数值字段的缺失值
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [121]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

在像这样的统计和机器学习的例子中，一个典型的任务是根据数据中的特征来预测乘客是否能幸存下来。将模型拟合到训练数据集上，然后在样本外测试数据集上进行评估。

用Age作为预测，但它缺少数据。有很多方法可以进行缺失数据插补（imputation），但做一个简单的插补，并使用训练数据集的中间值填充两个表中的空值：

In [122]:
impute_value = train['Age'].median()

train['Age'] = train['Age'].fillna(impute_value)

test['Age'] = test['Age'].fillna(impute_value)

现在明确模型。添加了一列IsFemale作为'Sex'列的编码版本：

In [124]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)

test['IsFemale'] = (test['Sex'] == 'female').astype(int)

然后决定一些模型变量并创建NumPy数组：

In [126]:
predictors = ['Pclass', 'IsFemale', 'Age']

X_train = train[predictors].values

X_test = test[predictors].values

y_train = train['Survived'].values

X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [127]:
y_train[:5]

array([0, 1, 1, 1, 0])

这不一定是一个好的模型，这些功能也不一定是正确的设计。接着使用scikit-learn的`LogisticRegression`模型创建一个模型实例：

In [131]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs')

与statsmodels类似，可以使用模型的fit方法在训练数据上拟合模型：

In [132]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

现在，可以使用`model.predict`为测试数据集形成预测：

In [133]:
y_predict = model.predict(X_test)

y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0])

如果拥有测试数据集的真实值，则可以计算精度百分比或其他一些错误指标：
```
(y_true == y_predict).mean()
```
实际上，模型训练中经常存在许多附加的复杂层次。许多模型具有可以调整的参数，并且存在可用于参数调整的交叉验证等技术以避免过度拟合训练数据。这通常可以在新数据上产生更好的预测性能或稳健性。

交叉验证通过分割训练数据来模拟样本外预测。基于像均方误差之类的模型准确度分数，可以对模型参数执行网格搜索。一些模型，如逻辑回归，具有内置交叉验证的估计类。例如，`LogisticRegressionCV`类可以与一个参数一起使用，该参数表示网格搜索在模型正则化参数C上的细致度：

In [136]:
from sklearn.linear_model import LogisticRegressionCV

model_cv = LogisticRegressionCV(10, cv=3)

model_cv.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=3, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

要手动进行交叉验证，可以使用`cross_val_score`帮助函数，该函数处理数据拆分过程。例如，为了用模型与训练数据的四个非重叠分割进行交叉验证，可以这样做：

In [138]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(C=10, solver='lbfgs')

scores = cross_val_score(model, X_train, y_train, cv=4)

scores

array([0.77232143, 0.80269058, 0.77027027, 0.78828829])

默认评分指标是依赖于模型的，但可以选择明确的评分函数。经过交叉验证的模型需要更长时间的训练，但通常可以产生更好的模型性能。